In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

import torch
from torch import nn
import torch.nn.functional as F

drive.mount('/content/gdrive', force_remount=True)
filepath = '/content/gdrive/MyDrive' + '/Colab Notebooks/csv/'
mnist = np.load(filepath + 'mnist.npz')

x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/gdrive
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [2]:
from torch.utils.data import TensorDataset, DataLoader

# Train Data Set
train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

# Test Data Set
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 512, bias=True)
        self.fc2 = nn.Linear(512, 512, bias=True)
        self.fc3 = nn.Linear(512, 512, bias=True)
        self.fc4 = nn.Linear(512, 512, bias=True) 
        self.fc5 = nn.Linear(512, 10, bias=True)

    def forward(self, x):
        x = F.relu(self.fc1(x)) # layer 1
        x = F.relu(self.fc2(x)) # layer 2
        x = F.relu(self.fc3(x)) # layer 3
        x = F.relu(self.fc4(x)) # layer 4
        x = self.fc5(x) # layer 5

        return x

In [5]:
from torch.optim import SGD

model = MLP().to(device)
criterion  = nn.CrossEntropyLoss().to(device)

opti = SGD(model.parameters(), lr=1e-3)

In [6]:
epoch = 10

losses = []
for i in range(epoch):
    correct = 0
    for data, target in train_dataloader:
        data = data.view(-1, 28*28).to(device)
        target = target.to(device)

        output = model(data)
        loss = criterion(output, target)

        opti.zero_grad()
        loss.backward()
        opti.step()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
    print(100.0 * correct / len(train_dataloader.dataset))

# GPU 35초
# CPU 1분 49초

22.246666666666666
49.085
62.32
76.27333333333333
83.79833333333333
86.98166666666667
88.56166666666667
89.75333333333333
90.45833333333333
91.04166666666667
